In [1]:
!pip -q install yfinance pandas numpy plotly flask gunicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 603.5 kB/s eta 0:00:00


In [2]:
import pandas as pd, numpy as np, yfinance as yf
from heapq import heappush, heappushpop
from typing import List, Tuple

def fetch_prices(tickers: List[str], period="1d", interval="5m"):
    """
    Returns (closes_df, volumes_df) indexed by datetime.
    Works with single or multiple tickers.
    """
    data = yf.download(
        tickers=tickers, period=period, interval=interval,
        auto_adjust=True, progress=False, threads=True
    )
    # Normalize to DataFrames with columns as tickers
    if isinstance(data.columns, pd.MultiIndex):
        closes = data["Close"].copy()
        volumes = data["Volume"].copy()
    else:  # single ticker
        closes = data["Close"].to_frame(name=tickers[0])
        volumes = data["Volume"].to_frame(name=tickers[0])
    return closes.dropna(how="all"), volumes.dropna(how="all")

def top_n_by_percent_change(closes: pd.DataFrame, n: int=5) -> List[Tuple[float,str]]:
    """
    Keeps a min-heap of the top N percent changes from first to last price today.
    Returns sorted list (largest first): [(pct_change, 'TICKER'), ...]
    """
    heap = []
    for t in closes.columns:
        s = closes[t].dropna()
        if len(s) < 2:
            continue
        change = (s.iloc[-1] - s.iloc[0]) / s.iloc[0] * 100.0
        item = (float(change), t)
        if len(heap) < n:
            heappush(heap, item)
        else:
            if item[0] > heap[0][0]:
                heappushpop(heap, item)
    return sorted(heap, reverse=True)

def top_n_by_volume(volumes: pd.DataFrame, n: int=5) -> List[Tuple[float,str]]:
    """
    Top N by **total** intraday volume using a min-heap.
    """
    heap = []
    for t in volumes.columns:
        total_vol = float(volumes[t].dropna().sum())
        item = (total_vol, t)
        if len(heap) < n:
            heappush(heap, item)
        else:
            if item[0] > heap[0][0]:
                heappushpop(heap, item)
    return sorted(heap, reverse=True)


In [3]:
# US examples:
tickers = ["AAPL","MSFT","AMZN","GOOGL","META","NVDA","TSLA","AMD","NFLX","INTC"]

# For Indian stocks, add the .NS suffix, e.g.:
# tickers = ["RELIANCE.NS","TCS.NS","HDFCBANK.NS","INFY.NS","ITC.NS","SBIN.NS","LT.NS","ICICIBANK.NS"]

closes, volumes = fetch_prices(tickers, period="1d", interval="5m")
print("Rows:", len(closes), "Cols:", len(closes.columns))

print("\nTop 5 by % change:")
print(top_n_by_percent_change(closes, n=5))

print("\nTop 5 by total volume:")
print(top_n_by_volume(volumes, n=5))


Rows: 1 Cols: 10

Top 5 by % change:
[]

Top 5 by total volume:
[(22517002.0, 'INTC'), (4830190.0, 'NVDA'), (2408511.0, 'AMD'), (1833220.0, 'TSLA'), (1192182.0, 'AAPL')]


In [4]:
import plotly.graph_objects as go

ticker_to_plot = tickers[0]
s = closes[ticker_to_plot].dropna()
fig = go.Figure(data=go.Scatter(x=s.index, y=s.values, mode="lines", name=ticker_to_plot))
fig.update_layout(title=f"Price — {ticker_to_plot}", xaxis_title="Time", yaxis_title="Price")
fig.show()


In [7]:
import os, textwrap, json, zipfile, pathlib

root = "/content/stock-heap-web"
tpl = os.path.join(root, "templates")
os.makedirs(tpl, exist_ok=True)

# ---------- app.py ----------
app_py = r'''
import os, time
from datetime import datetime, timezone
from heapq import heappush, heappushpop
from typing import List, Tuple
import pandas as pd
import yfinance as yf
from flask import Flask, request, jsonify, render_template

app = Flask(__name__)

DEFAULT_TICKERS = os.environ.get("TICKERS", "AAPL,MSFT,AMZN,GOOGL,META,NVDA,TSLA,AMD,NFLX,INTC").split(",")
CACHE_SECONDS = int(os.environ.get("CACHE_SECONDS", "60"))

_last_fetch = 0.0
_cached_closes = None
_cached_vols = None
_cached_meta = {}

def fetch_prices(tickers: List[str], period="1d", interval="5m"):
    data = yf.download(
        tickers=tickers, period=period, interval=interval,
        auto_adjust=True, progress=False, threads=True
    )
    if isinstance(data.columns, pd.MultiIndex):
        closes = data["Close"].copy()
        volumes = data["Volume"].copy()
    else:
        # single ticker case
        t = tickers[0]
        closes = data["Close"].to_frame(name=t)
        volumes = data["Volume"].to_frame(name=t)
    return closes.dropna(how="all"), volumes.dropna(how="all")

def ensure_data(tickers: List[str], period="1d", interval="5m"):
    global _last_fetch, _cached_closes, _cached_vols, _cached_meta
    now = time.time()
    tickers = [t.strip() for t in tickers if t.strip()]
    if not tickers:
        tickers = DEFAULT_TICKERS

    if (now - _last_fetch) > CACHE_SECONDS or _cached_closes is None:
        closes, vols = fetch_prices(tickers, period=period, interval=interval)
        _cached_closes, _cached_vols = closes, vols
        _cached_meta = {"tickers": tickers, "period": period, "interval": interval, "fetched_at": datetime.now(timezone.utc).isoformat()}
        _last_fetch = now
    return _cached_closes, _cached_vols

def top_n_by_percent_change(closes: pd.DataFrame, n: int=5):
    heap = []
    for t in closes.columns:
        s = closes[t].dropna()
        if len(s) < 2:
            continue
        change = (s.iloc[-1] - s.iloc[0]) / s.iloc[0] * 100.0
        item = (float(change), t)
        if len(heap) < n:
            heappush(heap, item)
        else:
            if item[0] > heap[0][0]:
                heappushpop(heap, item)
    out = sorted(heap, reverse=True)
    return [{"ticker": t, "value": round(v, 2)} for (v, t) in out]

def top_n_by_volume(vols: pd.DataFrame, n: int=5):
    heap = []
    for t in vols.columns:
        v = vols[t].dropna()
        if v.empty:
            continue
        total_vol = float(v.sum())
        item = (total_vol, t)
        if len(heap) < n:
            heappush(heap, item)
        else:
            if item[0] > heap[0][0]:
                heappushpop(heap, item)
    out = sorted(heap, reverse=True)
    return [{"ticker": t, "value": int(v)} for (v, t) in out]

@app.route("/")
def index():
    return render_template("index.html", default_tickers=",".join(DEFAULT_TICKERS))

@app.route("/api/top")
def api_top():
    metric = request.args.get("metric", "change").lower()
    n = int(request.args.get("n", "5"))
    tickers_param = request.args.get("tickers", "")
    tickers = [t.strip() for t in tickers_param.split(",") if t.strip()] or DEFAULT_TICKERS
    interval = request.args.get("interval", "5m")
    period = request.args.get("period", "1d")

    closes, vols = ensure_data(tickers, period=period, interval=interval)

    if metric == "volume":
        top = top_n_by_volume(vols, n=n)
        label = "total_volume"
    else:
        top = top_n_by_percent_change(closes, n=n)
        label = "pct_change"

    return jsonify({
        "metric": metric,
        "label": label,
        "top": top,
        "meta": {"tickers": tickers, "period": period, "interval": interval}
    })

@app.route("/api/series")
def api_series():
    ticker = request.args.get("ticker")
    if not ticker:
        return jsonify({"error": "ticker required"}), 400
    tickers_param = request.args.get("tickers", "")
    tickers = [t.strip() for t in tickers_param.split(",") if t.strip()] or DEFAULT_TICKERS
    interval = request.args.get("interval", "5m")
    period = request.args.get("period", "1d")

    closes, _ = ensure_data(tickers, period=period, interval=interval)
    if ticker not in closes.columns:
        return jsonify({"error": "ticker not in data"}), 404
    s = closes[ticker].dropna()
    series = [{"t": ts.isoformat(), "price": float(v)} for ts, v in zip(s.index, s.values)]
    return jsonify({"ticker": ticker, "series": series})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=int(os.environ.get("PORT", 5000)))
'''

# ---------- templates/index.html ----------
index_html = r'''
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Stock Market Data Analysis with Heaps</title>
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet">
  <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
  <style>
    body { background:#0b1220; color:#e9eef7; }
    .card { background:#111a2e; border:1px solid #2a3550; border-radius:1rem; }
    .badge-soft { background:#1b2945; border:1px solid #2a3550; }
    .box-border { border:2px solid #2a3550; border-radius:1rem; padding:1rem; }
    a, a:hover { color:#9dc1ff; }
    .table-dark tbody tr { cursor:pointer; }
  </style>
</head>
<body class="p-3">
  <div class="container-lg">
    <div class="d-flex justify-content-between align-items-center mb-3">
      <h2 class="m-0">Stock Market Data Analysis <span class="badge badge-soft ms-2">Heaps</span></h2>
      <div>Built with Flask + yfinance + Plotly</div>
    </div>

    <div class="card p-3 mb-3">
      <div class="row g-2">
        <div class="col-md-6">
          <label class="form-label">Tickers (comma-separated)</label>
          <input id="tickers" class="form-control" value="{{ default_tickers }}">
          <div class="form-text">For NSE use .NS (e.g., RELIANCE.NS,TCS.NS)</div>
        </div>
        <div class="col-md-2">
          <label class="form-label">Top N</label>
          <input id="topn" type="number" min="1" max="20" value="5" class="form-control">
        </div>
        <div class="col-md-2">
          <label class="form-label">Metric</label>
          <select id="metric" class="form-select">
            <option value="change" selected>% Change</option>
            <option value="volume">Total Volume</option>
          </select>
        </div>
        <div class="col-md-2">
          <label class="form-label">Interval</label>
          <select id="interval" class="form-select">
            <option>5m</option>
            <option>15m</option>
            <option>30m</option>
            <option>60m</option>
          </select>
        </div>
      </div>
      <div class="mt-3">
        <button id="refresh" class="btn btn-primary me-2">Refresh Now</button>
        <span class="text-muted">Auto-refresh every 15s</span>
      </div>
    </div>

    <div class="row g-3">
      <div class="col-lg-5">
        <div class="card p-3">
          <h5 class="mb-3">Top <span id="titleN">5</span> (<span id="titleMetric">% Change</span>)</h5>
          <div class="table-responsive">
            <table class="table table-dark table-striped align-middle" id="topTable">
              <thead><tr><th>#</th><th>Ticker</th><th class="text-end">Value</th></tr></thead>
              <tbody></tbody>
            </table>
          </div>
          <div class="small text-secondary">Click a row to load its chart.</div>
        </div>
      </div>
      <div class="col-lg-7">
        <div class="card p-3">
          <h5 id="chartTitle" class="mb-3">Price</h5>
          <div id="chart" style="height:420px;"></div>
        </div>
      </div>
    </div>

    <div class="mt-4 box-border small">
      <b>How it works:</b> The backend fetches prices with <code>yfinance</code>, keeps a fixed-size heap
      to track the top N tickers by % change or volume, and serves JSON to this page. The table updates every 15s.
    </div>
  </div>

<script>
let timer=null, currentTicker=null;

function params() {
  return {
    tickers: document.getElementById("tickers").value,
    n: document.getElementById("topn").value,
    metric: document.getElementById("metric").value,
    interval: document.getElementById("interval").value
  };
}

async function loadTop() {
  const p = params();
  const url = `/api/top?tickers=${encodeURIComponent(p.tickers)}&n=${p.n}&metric=${p.metric}&interval=${p.interval}`;
  const r = await fetch(url);
  const data = await r.json();
  document.getElementById("titleN").textContent = p.n;
  document.getElementById("titleMetric").textContent = (p.metric === "volume") ? "Total Volume" : "% Change";

  const tbody = document.querySelector("#topTable tbody");
  tbody.innerHTML = "";
  data.top.forEach((row, i) => {
    const tr = document.createElement("tr");
    tr.innerHTML = `<td>${i+1}</td><td>${row.ticker}</td>
                    <td class="text-end">${row.value.toLocaleString()}</td>`;
    tr.onclick = () => loadSeries(row.ticker);
    tbody.appendChild(tr);
  });

  if (!currentTicker && data.top.length) {
    loadSeries(data.top[0].ticker);
  }
}

async function loadSeries(ticker) {
  currentTicker = ticker;
  const p = params();
  const url = `/api/series?ticker=${encodeURIComponent(ticker)}&tickers=${encodeURIComponent(p.tickers)}&interval=${p.interval}`;
  const r = await fetch(url);
  const data = await r.json();
  if (data.error) return;

  const x = data.series.map(d => d.t);
  const y = data.series.map(d => d.price);
  const trace = { x, y, mode: "lines", name: ticker };
  const layout = { margin:{t:30,r:10,b:40,l:50}, paper_bgcolor:"#111a2e", plot_bgcolor:"#111a2e",
                   font:{color:"#e9eef7"}, xaxis:{title:"Time"}, yaxis:{title:"Price"} };
  Plotly.newPlot("chart", [trace], layout, {displayModeBar:false});
  document.getElementById("chartTitle").textContent = `Price — ${ticker}`;
}

document.getElementById("refresh").onclick = loadTop;
loadTop();
timer = setInterval(loadTop, 15000);
</script>
</body>
</html>
'''

# ---------- requirements.txt ----------
requirements_txt = """Flask==3.0.2
yfinance==0.2.40
pandas==2.2.2
numpy==1.26.4
plotly==5.22.0
gunicorn==21.2.0
"""

# Write files
with open(os.path.join(root, "app.py"), "w") as f: f.write(textwrap.dedent(app_py).strip()+"\n")
with open(os.path.join(tpl, "index.html"), "w") as f: f.write(index_html)
with open(os.path.join(root, "requirements.txt"), "w") as f: f.write(requirements_txt)

# Zip it
zip_path = "/content/stock-heap-web.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for p in pathlib.Path(root).rglob("*"):
        z.write(p, p.relative_to(root))

print("Wrote project to", root)
print("Zipped ->", zip_path)


Wrote project to /content/stock-heap-web
Zipped -> /content/stock-heap-web.zip


In [8]:
from google.colab import files
files.download("/content/stock-heap-web.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>